In [1]:
import numpy as np
import pandas as pd
import heapq
from collections import Counter
import copy

Firstly, import the package needed in this project. Then import the data from a csv file which is copied from the question. As is shown in the file, the data is composed of 4 attributes and 1 label('A','B' or 'C'). Then shuffle the data and randomly select training data and testing data from the dataset. The size of the training/testing data can be modified using the variable testsize

In [2]:
df = pd.read_csv('Data.csv',header=None,dtype=str)[0].str.split(',',expand=True)
df_array = df[:].values
df_x = df_array[:,:4].astype(np.float64)# Attributes
df_y = df_array[:,-1]#labels
index_shuffle = np.arange(len(df_y)) 
np.random.shuffle(index_shuffle)#shuffle the data
testsize=30
xtrain = df_x[index_shuffle[:-testsize]]#Randomly select traing set and testing set
ytrain = df_y[index_shuffle[:-testsize]]
xtest = df_x[index_shuffle[-testsize:]]
ytest = df_y[index_shuffle[-testsize:]]

Then define the similarity(distance) between data instances. Here we consider the Euclidean distance which means:

$$distance=\sqrt{\sum_{i=1}^{4}(x_{1i}-x_{2i})^2}$$

The instances with higher distances are less similar.

In [3]:
def dis(xtest,xtrain):
    p=2
    return ((xtest-xtrain)**p).sum(axis=1)**(1/p)

Next a function to find k nearest instance is defined below. For an instance from testing set(xtest), the difference between it and all the training instances(xtrain) are calculated. Then the index of instances with k smallest distances(k need input). Then function will return all the k instances.

In [4]:
def Knearest(xtrain,xtest,k):
    p=2
    distance=dis(xtest,xtrain)
    index_Ksmall = distance.argsort()[:k]
    Knearest=xtrain[:k]
    return Knearest

For example, the code below return the 30 nearest instance from the first instance in testing data.

In [5]:
Knearest(xtrain,xtest[0],30)

array([[5. , 2. , 3.5, 1. ],
       [6.7, 3.3, 5.7, 2.5],
       [5.6, 2.7, 4.2, 1.3],
       [6.7, 3.1, 4.7, 1.5],
       [6.9, 3.1, 5.1, 2.3],
       [5.9, 3. , 4.2, 1.5],
       [5.6, 2.9, 3.6, 1.3],
       [6.2, 3.4, 5.4, 2.3],
       [5.1, 3.8, 1.5, 0.3],
       [4.9, 2.5, 4.5, 1.7],
       [6.5, 3.2, 5.1, 2. ],
       [5.1, 3.7, 1.5, 0.4],
       [6.3, 3.4, 5.6, 2.4],
       [4.4, 3. , 1.3, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [6.4, 3.2, 5.3, 2.3],
       [4.9, 3.1, 1.5, 0.1],
       [5.8, 2.7, 5.1, 1.9],
       [5.4, 3.9, 1.3, 0.4],
       [7.3, 2.9, 6.3, 1.8],
       [4.8, 3.4, 1.6, 0.2],
       [5.8, 2.7, 4.1, 1. ],
       [4.4, 3.2, 1.3, 0.2],
       [5.9, 3. , 5.1, 1.8],
       [6.7, 3. , 5.2, 2.3],
       [4.6, 3.6, 1. , 0.2],
       [7.7, 3.8, 6.7, 2.2],
       [7.7, 2.6, 6.9, 2.3],
       [6.3, 2.7, 4.9, 1.8],
       [6.3, 2.3, 4.4, 1.3]])

Next the prediction data is defined. This KNN prediction function contains following steps:
1. For the first instance in testing data(xtest), calculate the distances between all the training data(xtrain)
2. Find the k nearest instances and find their classes from ytrain.
3. Calculate the counts of the classes that show in the k nearest instances, and find the class with highest frequency as the prediction result.
4. Repeat for rest of testing data and return the prediction result.

With the prediction result we can calculate the accuracy, which is:
$$accuracy = \frac{count_{match}}{count_{testing}} $$

In [6]:
def prediction(xtrain,ytrain,xtest,p):
    k=12
    y_predict=[]
    for i in range(len(xtest)):
        distance=dis(xtest[i],xtrain)
        index_Ksmall = distance.argsort()[:k]
        allClass = np.unique(ytrain[list(index_Ksmall)])
        
        Class_counts = Counter(allClass)
        finalClass = Class_counts.most_common(1)[0][0]
        y_predict.append(finalClass)
    return np.array(y_predict)

def accuracy(prediction_y,y_test):
    return (prediction_y==y_test).sum()/len(y_test)

For example, the prediction result is shown below and the predict accuracy is 0.97

In [7]:
prediction_y=prediction(xtrain,ytrain,xtest,2)
print(prediction_y)

['C' 'C' 'A' 'A' 'A' 'B' 'B' 'C' 'B' 'A' 'C' 'C' 'C' 'B' 'B' 'A' 'A' 'A'
 'B' 'B' 'B' 'A' 'A' 'A' 'B' 'B' 'B' 'C' 'B' 'B']


In [8]:
accuracy(prediction_y,ytest)

0.9666666666666667

Here the Knearest function is modified that the output will be the mean result of the k nearest instances instead of all the instances. The result is calculated as:
$$x_{mean,i}=\frac{\sum x_i}{k}$$

In [9]:
def Knearest2(xtrain,xtest,k):
    p=2
    distance=((xtest-xtrain)**p).sum(axis=1)**(1/p)
    index_Ksmall = distance.argsort()[:k]
    Knearest=xtrain[:k].mean(axis=0)
    return Knearest

An example is shown below.

In [10]:
Knearest2(xtrain,xtest[0],30)

array([5.86333333, 3.09333333, 3.93333333, 1.35666667])

This is the normalization data. Each attribute is rescaled to the $[0,1]$ section, which means:
$$x_{rescale}=\frac{x-x_{min}}{x_{max}-x_{min}}$$

In [11]:
def normalize(xtrain,xtest):
    t1=copy.deepcopy(xtrain[:,:])
    t2=copy.deepcopy(xtest[:,:])
    for i in range(xtest.shape[1]):
        t2[:,i]=(t2[:,i]-t1[:,i].min())/(t1[:,i].max()-t1[:,i].min())
    return t2

An example is shown below. What should be noticed is that the accuracy is not necessarily higher after the normalization because the optimal weights for the attributes are equal.

In [12]:
nxtrain = normalize(xtrain,xtrain)
nxtest = normalize(xtrain,xtest)
y_pre2=prediction(nxtrain,ytrain,nxtest,2)
accuracy(y_pre2,ytest)

0.9666666666666667

A new distance calculation function is defined as:
$$distance=(\sum_{i=1}^{4}|(x_{1i}-x_{2i})^p|)^\frac{1}{p}$$
Where p can be customized. When $p=2$, the distance will be Euclidean distance

In [13]:
def disp(xtest,xtrain,p):
    return abs((xtest-xtrain)**p).sum(axis=1)**(1/p)

As is shown below, for different p, the distances are different.

In [14]:
for p in (1,2,3,4,5,6):
    print(disp(xtest[0],xtrain[:4],p))

[6.7 2.2 4.4 2.8]
[3.73764632 1.12249722 2.68700577 1.64316767]
[3.19586053 0.90775197 2.34577481 1.4702278 ]
[2.99014506 0.82256258 2.20546887 1.42430505]
[2.88354549 0.77924347 2.12894633 1.40928563]
[2.81803146 0.75418843 2.08053196 1.40376989]
